In [1]:
import pandas as pd
from lxml import html
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]
    return ' '.join(words)

def tokenize(text):
    words = [word.strip(punct) for word in text.lower().split()]
    return ' '.join(words)

# Preprocessing

In [69]:
corpus_xml = html.fromstring(open('data/paraphrases.xml', 'rb').read())

texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1': texts_1, 'text_2': texts_2, 'label': classes})

In [70]:
data.head(10)

,label,text_1,text_2
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...
5,1,Приставы соберут отпечатки пальцев российских ...,Приставы снимут отпечатки пальцев у злостных н...
6,-1,На саратовского дебошира с борта самолета Моск...,Саратовский дебошир отказывается возвращаться ...
7,0,ЦИК хочет отказаться от электронной системы по...,ЦИК может отказаться от электронных средств по...
8,-1,Суд Петербурга оставил на потом дело о гибели ...,Лондонский Гайд-парк - это не место для митинг...
9,-1,Страны ОПЕК сократили добычу нефти на 1 млн ба...,Обама продлил полномочия НАСА по сотрудничеств...


In [71]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

In [72]:
data.head(10)

,label,text_1,text_2,text_1_norm,text_2_norm
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,москва сирия вернуться 2 самолёт мчс россиянин...,самолёт мчс вывезти россиянин разрушить сирия
5,1,Приставы соберут отпечатки пальцев российских ...,Приставы снимут отпечатки пальцев у злостных н...,пристав собрать отпечаток палец российский дол...,пристав снять отпечаток палец злостный неплате...
6,-1,На саратовского дебошира с борта самолета Моск...,Саратовский дебошир отказывается возвращаться ...,саратовский дебошир борт самолёт москва хургад...,саратовский дебошир отказываться возвращаться ...
7,0,ЦИК хочет отказаться от электронной системы по...,ЦИК может отказаться от электронных средств по...,цик хотеть отказаться электронный система подс...,цик отказаться электронный средство подсчёт голос
8,-1,Суд Петербурга оставил на потом дело о гибели ...,Лондонский Гайд-парк - это не место для митинг...,суд петербург оставить дело гибель подросток п...,лондонский гайд-парк это место митинг прежде парк
9,-1,Страны ОПЕК сократили добычу нефти на 1 млн ба...,Обама продлил полномочия НАСА по сотрудничеств...,страна опека сократить добыча нефть 1 миллион ...,обама продлить полномочие наса сотрудничество ...


# Обучение моделей

Для обучения этих моделей можете воспользоваться корпусом новостных текстов, с которым мы работали на семинаре. А можете использовать любой другой корпус (сами тексты соревнования использовать не надо).



In [31]:
news = pd.read_csv('data/news_texts.csv')
news.dropna(inplace=True)

In [35]:
news.head()

,content,content_norm
0,Канцлер Германии Ангела Меркель в ходе брифинг...,канцлер германия ангел меркель ход брифинг пре...
1,Российские и белорусские войска успешно заверш...,российский белорусский войско успешно завершит...
2,"Дзюба, Шатов и Анюков оказались не нужны «Зени...",дзюба шат анюк оказаться нужный зенит российск...
3,"В Испанию без фанатов\nПожалуй, главной пятнич...",испания фанат пожалуй главный пятничный новост...
4,"Постпред России при ООН Виталий Чуркин, говоря...",постпред россия оон виталий чуркин говорить ве...


In [32]:
cv = CountVectorizer(min_df=3, max_df=0.4, max_features=1000)
X = cv.fit_transform(news['content_norm'])

In [33]:
X.shape

(7212, 1000)

In [36]:
tfidf = TfidfVectorizer(min_df=3, max_df=0.4, max_features=1000)
X_tf = tfidf.fit_transform(news['content_norm'])

# Векторизация

Преобразуйте тексты в векторы в каждой паре 4 методами  - SVD, NMF, Word2Vec, Fastext. Для SVD и NMF сделайте две пары векторов - через TfidfVectorizer и CountVectorizer. 

### SVD

In [102]:
svd = TruncatedSVD(50)
svd.fit(X)

TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
       random_state=None, tol=0.0)

In [103]:
text_1_cv = cv.transform(data['text_1_norm'])
text_2_cv = cv.transform(data['text_2_norm'])

In [104]:
text_1_tf = tfidf.transform(data['text_1_norm'])
text_2_tf = tfidf.transform(data['text_2_norm'])

In [105]:
X_text_1 = svd.fit_transform(text_1_cv)
X_text_2 = svd.fit_transform(text_2_cv)

X_svd_cv = np.concatenate([X_text_1, X_text_2], axis=1)

In [106]:
svd_tf = TruncatedSVD(50)
svd_tf.fit(X_tf)

TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
       random_state=None, tol=0.0)

In [107]:
X_text_1 = svd_tf.fit_transform(text_1_tf)
X_text_2 = svd_tf.fit_transform(text_2_tf)

X_svd_tf = np.concatenate([X_text_1, X_text_2], axis=1)

### NMF

In [50]:
nmf = NMF(50)
nmf.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=50, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [51]:
X_text_1_nmf = nmf.transform(text_1_cv)
X_text_2_nmf = nmf.transform(text_2_cv)

X_nmf_cv = np.concatenate([X_text_1_nmf, X_text_2_nmf], axis=1)

In [52]:
nmf.fit(X_tf)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=50, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [53]:
X_text_1_nmf = nmf.transform(text_1_tf)
X_text_2_nmf = nmf.transform(text_2_tf)

X_nmf_tf = np.concatenate([X_text_1_nmf, X_text_2_nmf], axis=1)

### Word2Vec

Для word2vec сделайте две пары векторов - с взвешиванием по tfidf и без.

In [55]:
w2v = gensim.models.Word2Vec([text.split() for text in news['content_norm']], size=50, sg=1)

In [64]:
def get_embedding(text, model, dim, tf=False):
    
    text = text.split()
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i, word in enumerate(words):
        try:
            if tf:
                w = tfidf.vocabulary_[word]
            else:
                w = (words[word]/total)
        
            v = model[word]
            vectors[i] = v * w
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [57]:
dim = 50
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim)

/home/socur/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [58]:
X_w2v = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)

In [65]:
dim = 50
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim, tf=True)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim, tf=True)

/home/socur/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [66]:
X_w2v_tf = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)

### Fastext

Для Fastext постройте две модели - без нормализации и с нормализацией, а через каждую модель постройте две пары векторов -  с взвешиванием по tfidf и без. 

In [67]:
# с нормализацией
fast_text = gensim.models.FastText([text.split() for text in news['content_norm']], size=50, min_n=4, max_n=8)

In [68]:
# без нормализации
fast_text_simple = gensim.models.FastText([text.split() for text in news['content']], size=50, min_n=4, max_n=8)

In [73]:
# без нормализации и взвешивания

dim = 50
data['text_1_notnorm'] = data['text_1'].apply(tokenize)
data['text_2_notnorm'] = data['text_2'].apply(tokenize)

X_text_1_ft = np.zeros((len(data['text_1_notnorm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_notnorm']), dim))

for i, text in enumerate(data['text_1_notnorm'].values):
    X_text_1_ft[i] = get_embedding(text, fast_text_simple, dim)
    
for i, text in enumerate(data['text_2_notnorm'].values):
    X_text_2_ft[i] = get_embedding(text, fast_text_simple, dim)
    
X_ft_simple = np.concatenate([X_text_1_ft, X_text_2_ft], axis=1)

/home/socur/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [74]:
# без нормализации и со взвешиванием

dim = 50
data['text_1_notnorm'] = data['text_1'].apply(tokenize)
data['text_2_notnorm'] = data['text_2'].apply(tokenize)

X_text_1_ft = np.zeros((len(data['text_1_notnorm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_notnorm']), dim))

for i, text in enumerate(data['text_1_notnorm'].values):
    X_text_1_ft[i] = get_embedding(text, fast_text_simple, dim, tf=True)
    
for i, text in enumerate(data['text_2_notnorm'].values):
    X_text_2_ft[i] = get_embedding(text, fast_text_simple, dim, tf=True)
    
X_ft_simple_tf = np.concatenate([X_text_1_ft, X_text_2_ft], axis=1)

/home/socur/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [76]:
# с нормализацией и без взвешивания

dim = 50
data['text_1_notnorm'] = data['text_1'].apply(tokenize)
data['text_2_notnorm'] = data['text_2'].apply(tokenize)

X_text_1_ft = np.zeros((len(data['text_1_notnorm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_notnorm']), dim))

for i, text in enumerate(data['text_1_notnorm'].values):
    X_text_1_ft[i] = get_embedding(text, fast_text, dim)
    
for i, text in enumerate(data['text_2_notnorm'].values):
    X_text_2_ft[i] = get_embedding(text, fast_text, dim)
    
X_ft = np.concatenate([X_text_1_ft, X_text_2_ft], axis=1)

/home/socur/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [77]:
# с нормализацией и взвешиванием

dim = 50
data['text_1_notnorm'] = data['text_1'].apply(tokenize)
data['text_2_notnorm'] = data['text_2'].apply(tokenize)

X_text_1_ft = np.zeros((len(data['text_1_notnorm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_notnorm']), dim))

for i, text in enumerate(data['text_1_notnorm'].values):
    X_text_1_ft[i] = get_embedding(text, fast_text, dim, tf=True)
    
for i, text in enumerate(data['text_2_notnorm'].values):
    X_text_2_ft[i] = get_embedding(text, fast_text, dim, tf=True)
    
X_ft_tf = np.concatenate([X_text_1_ft, X_text_2_ft], axis=1)

/home/socur/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


# Косинусная близость

У вас должно получиться 10 пар векторов для каждой строчки в датасете. Между векторами каждой пары вычислите косинусную близость (получится 10 чисел для каждой пары текстов). 

In [109]:
vectors = [X_svd_cv, X_svd_tf, X_nmf_cv, X_nmf_tf, X_w2v, X_w2v_tf, 
          X_ft_simple, X_ft_simple_tf, X_ft, X_ft_tf]

In [121]:
dist = {}

for i, texts in enumerate(vectors):
    text1 = texts[0]
    text2 = texts[1]
    dists = []
    
    for indx, pair in enumerate(text1):
        d = cosine_distances(text1[indx].reshape(1, -1), text2[indx].reshape(1, -1))[0]
        dists.append(d[0])
    
    dist[i] = dists

In [124]:
cos_dist = pd.DataFrame(dist)
cos_dist.head(10)

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,2.0,0.0,2.0
1,0.0,0.0,1.0,1.0,2.0,2.0,0.0,2.0,0.0,2.0
2,2.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,2.0
3,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
4,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0
5,0.0,0.0,1.0,1.0,0.0,2.0,2.0,2.0,0.0,0.0
6,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0
8,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,2.0,2.0
9,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,2.0


# Обучение модели

Постройте обучающую выборку из этих близостей. Обучите любую модель (Логрег, Рандом форест или что-то ещё) на этой выборке и оцените качество на кросс-валидации (используйте микросреднюю f1-меру).   

In [143]:
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import GridSearchCV, cross_val_score

In [128]:
y = data['label'].values

In [136]:
train_X, valid_X, train_y, valid_y = train_test_split(cos_dist, y[:100], random_state=42)
clf = LogisticRegression(C=1000, class_weight='balanced')
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(f1_score(valid_y, preds, average='micro'))

0.52
